# Setup

In [1]:
# os.environ['LC_ALL'] = 'C.UTF-8'
# os.environ['LANG'] = 'C.UTF-8'

In [2]:
!pip install pydub

Get MusicGen git repo

In [3]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft

  Cloning https://github.com/facebookresearch/audiocraft to /tmp/pip-install-un8ifbg1/audiocraft_4d9b0c07d85e488eb8680917cfcc27c0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft /tmp/pip-install-un8ifbg1/audiocraft_4d9b0c07d85e488eb8680917cfcc27c0
  Resolved https://github.com/facebookresearch/audiocraft to commit 72cb16f9fb239e9cf03f7bd997198c7d7a67a01c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.8 MB/s eta 0:00:00
  Prepari

Access to Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Imports

In [5]:
import os
import random
from pydub import AudioSegment

import soundfile as sf
import numpy as np

# Data Processing

Create auxiliary folders if necessary

In [6]:
new_folders = ["raw", "output", "musicgen_trainer_dir"]

for folder_name in new_folders:
  if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"'{folder_name}'folder successfully created!")

'raw'folder successfully created!
'output'folder successfully created!
'musicgen_trainer_dir'folder successfully created!


Create necessary folders in Drive

In [11]:
new_folders = ["raw", "output", "results"]

for folder_name in new_folders:
  file_path = os.path.join("/content/drive/MyDrive/ap-project/", folder_name)
  if not os.path.exists(file_path):
    os.makedirs(file_path)
    print(f"'{folder_name}'folder successfully created!")

'testing'folder successfully created!


In [12]:
raw = "/content/drive/MyDrive/ap-project/raw/desgarrada"
if not os.path.exists(raw):
   os.makedirs(raw)
   print(f"'{raw}'folder successfully created!")

output = "/content/drive/MyDrive/ap-project/output/desgarrada"
if not os.path.exists(output):
   os.makedirs(output)
   print(f"'{output}'folder successfully created!")

'/content/drive/MyDrive/ap-project/output/aaaa'folder successfully created!


Function to clean a directory

In [7]:
def clean_dir(dir_path):
  for file_name in os.listdir(dir_path):
    file_path = os.path.join(dir_path, file_name)
    if os.path.isfile(file_path):
      os.remove(file_path)
      print(f"File {file_path} was deleted")

Clean output directory

In [ ]:
clean_dir('/content/drive/MyDrive/ap-project/output')

File /content/drive/MyDrive/ap-project/output/other/segment_000.wav was deleted
File /content/drive/MyDrive/ap-project/output/other/segment_000.txt was deleted
File /content/drive/MyDrive/ap-project/output/other/segment_001.wav was deleted
File /content/drive/MyDrive/ap-project/output/other/segment_001.txt was deleted
File /content/drive/MyDrive/ap-project/output/other/segment_002.wav was deleted
File /content/drive/MyDrive/ap-project/output/other/segment_002.txt was deleted
File /content/drive/MyDrive/ap-project/output/other/segment_003.wav was deleted
File /content/drive/MyDrive/ap-project/output/other/segment_003.txt was deleted


In case raw and output are not in Drive

In [ ]:
!cp -r /content/raw/* "/content/drive/MyDrive/ap-project/raw"

In [ ]:
!cp -r /content/output/* "/content/drive/MyDrive/ap-project/output"

If you upload more audios to raw's folder, clean output's folder before running process_audios function

In [ ]:
clean_dir('/content/drive/MyDrive/ap-project/output')

Function to process audio data: segments audio in 30 seconds each and save .txt labels and set sample rate to 32000 Hz

In [8]:
def process_audios(file_path, output_dir, start_segment, segment_length=30): # 30 seconds
  # Load audio
  audio = AudioSegment.from_file(file_path)

  # Extract file name for .txt
  file_name = os.path.splitext(os.path.basename(file_path))[0]

  # Convert segment length to ms
  segment_length_ms = segment_length * 1000

  # Set the sample rate to 32000 Hz
  audio = audio.set_frame_rate(32000)

  # Calculate number of segments
  num_segments = (len(audio) + segment_length_ms - 1) // segment_length_ms

  for i in range(num_segments):
    start_time = i * segment_length_ms

    # Last segment
    if i == num_segments - 1:
      start_time = len(audio) - segment_length_ms

    end_time = start_time + segment_length_ms

    # Get segment
    segment = audio[start_time:end_time]
    print(f"i: {i} file: {file_path} - segment_{start_segment:03d}.wav")

    # Save segment
    segment.export(os.path.join(output_dir, f'segment_{start_segment:03d}.wav'), format='wav')

    # Save label
    with open(os.path.join(output_dir, f'segment_{start_segment:03d}.txt'), 'w') as f:
      f.write(file_name)
    start_segment +=1

  return start_segment

Process all audios

In [9]:
output_dir = '/content/drive/MyDrive/ap-project/output/desgarrada'
samples_dir = '/content/drive/MyDrive/ap-project/raw/desgarrada'

if not os.path.exists(output_dir):
  os.makedirs(output_dir)

start_segment = 0

for file_name in os.listdir(samples_dir):
  print(file_name)
  if file_name.endswith('.wav') or file_name.endswith('.mp3'):
    file_path = os.path.join(samples_dir, file_name)
    current_segment = process_audios(file_path, output_dir, start_segment, segment_length=30)
    start_segment = current_segment


desgarrada, adilia ribeiro .mp3
i: 0 file: /content/drive/MyDrive/ap-project/raw/desgarrada/desgarrada, adilia ribeiro .mp3 - segment_000.wav
i: 1 file: /content/drive/MyDrive/ap-project/raw/desgarrada/desgarrada, adilia ribeiro .mp3 - segment_001.wav
i: 2 file: /content/drive/MyDrive/ap-project/raw/desgarrada/desgarrada, adilia ribeiro .mp3 - segment_002.wav
i: 3 file: /content/drive/MyDrive/ap-project/raw/desgarrada/desgarrada, adilia ribeiro .mp3 - segment_003.wav
i: 4 file: /content/drive/MyDrive/ap-project/raw/desgarrada/desgarrada, adilia ribeiro .mp3 - segment_004.wav
desgarrada concertina, ruben ribeiro.mp3
i: 0 file: /content/drive/MyDrive/ap-project/raw/desgarrada/desgarrada concertina, ruben ribeiro.mp3 - segment_005.wav
i: 1 file: /content/drive/MyDrive/ap-project/raw/desgarrada/desgarrada concertina, ruben ribeiro.mp3 - segment_006.wav
i: 2 file: /content/drive/MyDrive/ap-project/raw/desgarrada/desgarrada concertina, ruben ribeiro.mp3 - segment_007.wav
i: 3 file: /content/

In [10]:
import librosa

output_dir = "/content/drive/MyDrive/ap-project/output/desgarrada"

for file_name in os.listdir(output_dir):
  if file_name.endswith('.wav'):
    file_path = os.path.join(output_dir, file_name)
    audio, sample_rate = librosa.load(file_path, sr=None)

    if audio.shape[0] == 32000 * 30:
      print(f"{file_name} has the correct shape: {audio.shape[0]}")
    else:
      print(f"{file_name} does not have the correct shape: {audio.shape[0]}")


segment_000.wav has the correct shape: 960000
segment_001.wav has the correct shape: 960000
segment_002.wav has the correct shape: 960000
segment_003.wav has the correct shape: 960000
segment_004.wav has the correct shape: 960000
segment_005.wav has the correct shape: 960000
segment_006.wav has the correct shape: 960000
segment_007.wav has the correct shape: 960000
segment_008.wav has the correct shape: 960000
segment_009.wav has the correct shape: 960000
segment_010.wav has the correct shape: 960000
segment_011.wav has the correct shape: 960000
segment_012.wav has the correct shape: 960000
segment_013.wav has the correct shape: 960000
segment_014.wav has the correct shape: 960000
segment_015.wav has the correct shape: 960000
segment_016.wav has the correct shape: 960000
segment_017.wav has the correct shape: 960000
segment_018.wav has the correct shape: 960000
segment_019.wav has the correct shape: 960000
segment_020.wav has the correct shape: 960000
segment_021.wav has the correct sh

# Training process

Get MusicGen Trainer

In [13]:
!cd "/content/musicgen_trainer_dir" && git clone https://github.com/chavinlo/musicgen_trainer.git

Cloning into 'musicgen_trainer'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 166 (delta 14), reused 19 (delta 6), pack-reused 136
Receiving objects: 100% (166/166), 10.53 MiB | 32.10 MiB/s, done.
Resolving deltas: 100% (93/93), done.


Install requirement

In [14]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.1/281.1 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00


In [15]:
import wandb

In [16]:
!python /content/musicgen_trainer_dir/musicgen_trainer/run.py --dataset_path /content/drive/MyDrive/ap-project/output/desgarrada --epochs 25 --batch_size 2

2024-05-17 15:16:43.920371: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 15:16:43.920422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 15:16:44.047440: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-17 15:16:46.716428: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/audiocraft/models/musicgen.py:80: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-small
  warnings.warn(
state

In [17]:
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch

Load model

In [18]:
model = musicgen.MusicGen.get_pretrained('small', device='cuda')
model.set_generation_params(duration=8)
model.lm.load_state_dict(torch.load('models/lm_final.pt'))

/usr/local/lib/python3.10/dist-packages/audiocraft/models/musicgen.py:80: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-small
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in

<All keys matched successfully>

Prompt

Clean results folder if necessary

In [ ]:
clean_dir("/content/drive/MyDrive/ap-project/results")

File /content/drive/MyDrive/ap-project/output/desgarrada/segment_000.wav was deleted
File /content/drive/MyDrive/ap-project/output/desgarrada/segment_000.txt was deleted
File /content/drive/MyDrive/ap-project/output/desgarrada/segment_001.wav was deleted
File /content/drive/MyDrive/ap-project/output/desgarrada/segment_001.txt was deleted
File /content/drive/MyDrive/ap-project/output/desgarrada/segment_002.wav was deleted
File /content/drive/MyDrive/ap-project/output/desgarrada/segment_002.txt was deleted
File /content/drive/MyDrive/ap-project/output/desgarrada/segment_003.wav was deleted
File /content/drive/MyDrive/ap-project/output/desgarrada/segment_003.txt was deleted
File /content/drive/MyDrive/ap-project/output/desgarrada/segment_004.wav was deleted
File /content/drive/MyDrive/ap-project/output/desgarrada/segment_004.txt was deleted
File /content/drive/MyDrive/ap-project/output/desgarrada/segment_005.wav was deleted
File /content/drive/MyDrive/ap-project/output/desgarrada/segment_

In [26]:
res = model.generate([
    'desgarrada, adilia ribeiro'
],
  progress=True)

sample_rate = 32000

audio_array = res.cpu().numpy()

audio_segment = AudioSegment(
    audio_array.tobytes(),
    frame_rate=sample_rate,
    sample_width=audio_array.dtype.itemsize,
    channels=1
)

output_dir = "/content/drive/MyDrive/ap-project/results"

existing_files = [f for f in os.listdir(output_dir) if f.startswith("audio") and f.endswith(".mp3")]
existing_numbers = [int(f[5:-4]) for f in existing_files if f[5:-4].isdigit()]
next_number = max(existing_numbers, default=0) + 1

output_path = os.path.join(output_dir, f"audio{next_number}.mp3")

audio_segment.export(output_path, format="mp3")

print(f"Audio saved at '{output_path}'")

display_audio(res, 32000)




Audio saved at '/content/drive/MyDrive/ap-project/results/audio8.mp3'
